# Data cleaning process for the Provider Suspended and Ineligible List
## Creates 2 final datasets that are saved to CSV to be fed directly to my Streamlit app

In [1]:
import pandas as pd
import time
import re



In [2]:
df = pd.read_csv("provider-suspended-and-ineligible-list-s-i-list.csv", encoding="windows-1252")


### Extracting addresses (with the help of Chat-GPT!)

I wanted to visualize the suspended providers' geographic distributions using zip codes and associated lattitude and longitude coordinates. First, I had to decide what to do with providers with multiple addresses, and wrote a function that extracts only the first address listed, and then extracts the zip code. However, I ran into the problem of 5 digit street addresses being encoded as zip codes, and Chat-GPT helped me with this revised implemention below that first checks if a 5 digit number is at the start of of the field (a street address), and if so, then checks to see if there is a second 5 digit number (the zip code).

Any additional addresses that come after the zip code of the 1st address (or the state code if the zip code is missing) are excluded and the resulting text populates a new '1st Address' column. Additional columns are created for City and State Code (which will be needed to find the missing zip codes).

In [3]:
import re

def extract_address_with_zip_or_state(address):
    # Search for patterns that might indicate a zip code or state code
    matches = re.findall(r'\b\d{5}\b', address)
    state_code_match = re.search(r'.*?\b([A-Z]{2})', address)
    
    if matches:
        # If there's more than one 5-digit sequence, assume the last one is the zip code
        if len(matches) > 1:
            zip_code = matches[-1]
            # Extract the part of the address up to and including the zip code
            zip_pos = address.rfind(zip_code)
            return address[:zip_pos + len(zip_code)]
        else:
            # Check if the first 5 digits are at the start (likely a street address) and a state code is present
            if address.startswith(matches[0]) and state_code_match:
                # Return the address up to and including the state code
                return state_code_match.group(0)
            else:
                # Otherwise, return the address up to and including the first (and only) zip code
                return address[:address.find(matches[0]) + len(matches[0])]
    elif state_code_match:
        # Return the address up to and including the state code if no zip code is found
        return state_code_match.group(0)
    else:
        # If no zip code or state code is found, return the original address
        return address

# Example usage
df['1st Address'] = df['Address(es)'].apply(extract_address_with_zip_or_state)

df['State Code'] = df['1st Address'].str.extract(r'.*?\b([A-Z]{2})\b', expand=False)

# Continue with city and zip code extraction as before
df['City'] = df['1st Address'].str.extract('([^,]+), [A-Z]{2},? ?\d*$', expand=False)
df['Zip Code'] = df['1st Address'].str.extract('(\d{5})$', expand=False)

# Handling special cases where the address might not follow the pattern
df['City'] = df['City'].fillna('Unknown')  # Filling NaN values for City with 'Unknown'
df['Zip Code'] = df['Zip Code'].fillna('Unknown')  # Filling NaN values for Zip Code with 'Unknown'


In [4]:
df

,Last Name,First Name,Middle Name,A/K/A-Also Known As\nD/B/A-Doing Business as,Address(es),Provider Type,License Number,Provider Number,Date of Suspension,Active Period,1st Address,State Code,City,Zip Code
0,111 Pharmacy,NaN,NaN,NaN,"111 West Beverly Blvd., Ste. B, Montebello, CA...",Pharmacy/Pharmacist,41023,"PHA410230, PHA393440, 1821137118",7/8/2016,Indefinitely effective,"111 West Beverly Blvd., Ste. B, Montebello, CA...",CA,Montebello,90640
1,"2nd Chance for Recovery, Inc.",NaN,NaN,NaN,"600 East 7th St., Los Angeles, CA, 90021",Entity/Corporation,NaN,"1003077710, 1033522974, 1346589777, 1760751606",05/23/2018,Indefinitely effective,"600 East 7th St., Los Angeles, CA, 90021",CA,Los Angeles,90021
2,3 Angeles Medical Clinic,NaN,NaN,NaN,"333 Wilkerson Ave., Stes. B & C, Perris, CA; 4...",Clinic,NaN,NaN,3/11/2009,Indefinitely effective,"333 Wilkerson Ave., Stes. B & C, Perris, CA",CA,Perris,Unknown
3,52 Street Pharmacy,NaN,NaN,NaN,"5231 El Cajon Blvd., San Diego, CA,",Pharmacy,NaN,"PHA118810, PHA200040, PHA217110, CGP14678",12/26/1990,Indefinitely effective,"5231 El Cajon Blvd., San Diego, CA",CA,San Diego,Unknown
4,A & R Medical Transport,NaN,NaN,"Aghajanyan, Mike; Aghajanyan, Roland; Aghajany...","5300 Santa Monica Blvd., Ste. #216, Los Angele...",Medical Transportation/Owner,NaN,MTN01203F,7/4/2012,Indefinitely effective,"5300 Santa Monica Blvd., Ste. #216, Los Angele...",CA,Los Angeles,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21631,Wehrly,John,Dale,NaN,"154 Ely Street, Oceanside, CA, 92054",Registered Nurse,484699,NaN,06/15/2023,Indefinitely effective,"154 Ely Street, Oceanside, CA, 92054",CA,Oceanside,92054
21632,Weinberger,Alan,Walter,NaN,"P.O. Box 676, South Pasadena, CA, 91031",Physician,G 33359,"00G333590, 1124015011",05/18/2023,Indefinitely effective,"P.O. Box 676, South Pasadena, CA, 91031",CA,South Pasadena,91031
21633,Winters,Julie,Mae,NaN,"6024 Fyler Avenue, Saint Louis, MO, 63139",Registered Nurse,774207,NaN,06/16/2023,Indefinitely effective,"6024 Fyler Avenue, Saint Louis, MO, 63139",MO,Saint Louis,63139
21634,Dondiego,Madelaine,NaN,NaN,"Delano, CA",In-Home Supportive Services Provider,1336006,NaN,07/11/2023,Indefinitely effective,"Delano, CA",CA,Delano,Unknown


###  Adding missing Zip Codes

11876 of the 21,636 providers in the original file are missing zip codes in the address column. I discovered an API that I could search addresses with called Nomatim, but it doesn't work when apartment or suite numbers are included, so I had to strip everything from the street address to combine it with the 'City' and 'State Code' values to correctly query Nomatim's API for the missing 11876 zip codes. 

In [5]:
def clean_suite_apt_info(address):
    # Count the number of commas in the address
    
    parts = address.split(',')
    # Reconstruct the address without the part between the first and second comma
    street_address = parts[0] 
    
    return street_address

# Apply the function to clean apartment or suite info from "1st Address"
df['Street Address'] = df['1st Address'].apply(clean_suite_apt_info)



In [6]:
df.head(50)

,Last Name,First Name,Middle Name,A/K/A-Also Known As\nD/B/A-Doing Business as,Address(es),Provider Type,License Number,Provider Number,Date of Suspension,Active Period,1st Address,State Code,City,Zip Code,Street Address
0,111 Pharmacy,NaN,NaN,NaN,"111 West Beverly Blvd., Ste. B, Montebello, CA...",Pharmacy/Pharmacist,41023,"PHA410230, PHA393440, 1821137118",7/8/2016,Indefinitely effective,"111 West Beverly Blvd., Ste. B, Montebello, CA...",CA,Montebello,90640,111 West Beverly Blvd.
1,"2nd Chance for Recovery, Inc.",NaN,NaN,NaN,"600 East 7th St., Los Angeles, CA, 90021",Entity/Corporation,NaN,"1003077710, 1033522974, 1346589777, 1760751606",05/23/2018,Indefinitely effective,"600 East 7th St., Los Angeles, CA, 90021",CA,Los Angeles,90021,600 East 7th St.
2,3 Angeles Medical Clinic,NaN,NaN,NaN,"333 Wilkerson Ave., Stes. B & C, Perris, CA; 4...",Clinic,NaN,NaN,3/11/2009,Indefinitely effective,"333 Wilkerson Ave., Stes. B & C, Perris, CA",CA,Perris,Unknown,333 Wilkerson Ave.
3,52 Street Pharmacy,NaN,NaN,NaN,"5231 El Cajon Blvd., San Diego, CA,",Pharmacy,NaN,"PHA118810, PHA200040, PHA217110, CGP14678",12/26/1990,Indefinitely effective,"5231 El Cajon Blvd., San Diego, CA",CA,San Diego,Unknown,5231 El Cajon Blvd.
4,A & R Medical Transport,NaN,NaN,"Aghajanyan, Mike; Aghajanyan, Roland; Aghajany...","5300 Santa Monica Blvd., Ste. #216, Los Angele...",Medical Transportation/Owner,NaN,MTN01203F,7/4/2012,Indefinitely effective,"5300 Santa Monica Blvd., Ste. #216, Los Angele...",CA,Los Angeles,Unknown,5300 Santa Monica Blvd.
5,A&M Medical,NaN,NaN,"Jefferson, Michael Hall","3901 Ursula Ave., Unit #1, Los Angeles, CA",Durable Medical Equipment; Owner,NaN,NaN,6/17/2004,Indefinitely effective,"3901 Ursula Ave., Unit #1, Los Angeles, CA",CA,Los Angeles,Unknown,3901 Ursula Ave.
6,AAA Home Health Service,NaN,NaN,NaN,"4708 Crenshaw Blvd., Los Angeles, CA",Home Health Agency,HHA70096F,NaN,6/10/1987,Indefinitely effective,"4708 Crenshaw Blvd., Los Angeles, CA",CA,Los Angeles,Unknown,4708 Crenshaw Blvd.
7,Aachi,Venkat,NaN,NaN,"900 Kiely Blvd., Santa Clara, CA, 95051",Physician,85279,1003889502,06/04/2020,Deceased,"900 Kiely Blvd., Santa Clara, CA, 95051",CA,Santa Clara,95051,900 Kiely Blvd.
8,Aannestad,Liv,Kirsten,NaN,"14248 N. Ashbrook Dr., Apt. A, Fountain Hills,...",Registered Nurse,672291,NaN,02/09/2019,Indefinitely effective,"14248 N. Ashbrook Dr., Apt. A, Fountain Hills,...",AZ,Fountain Hills,85268,14248 N. Ashbrook Dr.
9,Aaron,Benjamin,L.,NaN,"10333 Vista Del Capitan, Lakeside, CA",Physician,C23473,NaN,7/27/2010,Indefinitely effective,"10333 Vista Del Capitan, Lakeside, CA",CA,Lakeside,Unknown,10333 Vista Del Capitan


### Attempting to query Nomatim's API efficiently

Nomatim's API will block you if you flood it with requests, so the only way to retrieve the zip codes is itteratively requesting them, which is very very inefficient when you have over 20 thousand rows. As I was searching for ways to speed up the process. Chat-GPT introduced me to ThreadPoolExecutor which I didn't know about before, which allows you to make API requests in parallel. If you expand the output of the cell below, you will see the result of print statements for 8,686 new zip code values that were returned from Nomatim's API, and you will notice that the index number does not always increase consecutively due to the multiple parallel requests being made to the API by the different workers in the ThreadPoolExecutor. 

However, even with this moderate speedup, the process was still very slow, and it took over an hour to run.

In [8]:

from concurrent.futures import ThreadPoolExecutor, as_completed
import requests
import time

# Function modified to include a rate limit
def get_zip_code(index, street_address, city, state_code):
    time.sleep(1)  # Ensuring at least 1 second between requests to respect usage policy
    address_query = f"{street_address}, {city}, {state_code}"
    params = {'q': address_query, 'format': 'json', 'addressdetails': 1}
    response = requests.get('https://nominatim.openstreetmap.org/search', params=params)
    if response.status_code == 200:
        data = response.json()
        if data:
            postal_code = data[0].get('address', {}).get('postcode')
            if postal_code and len(postal_code.strip()) == 5 and postal_code.isdigit():
                return index, postal_code
    return index, None

# Filter the DataFrame to only include rows with 'Unknown' zip codes
unknown_zip_df = df[df['Zip Code'] == 'Unknown']

# Create a thread pool and make requests in parallel
with ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(get_zip_code, index, row['Street Address'], row['City'], row['State Code'])
               for index, row in unknown_zip_df.iterrows()]
    
    for future in as_completed(futures):
        index, zip_code = future.result()
        if zip_code:
            print(index, zip_code)
            df.at[index, 'Zip Code'] = zip_code


2 92570
5 90008
6 90043
3 92115
4 90029
9 92040
13 95355
15 91701
17 93606
20 86442
23 92555
25 92395
31 47501
35 90057
38 93534
37 91606
39 70130
40 92115
41 94947
45 98416
47 94707
50 92394
51 92262
57 90035
58 90503
63 92660
65 92127
74 91352
75 90404
76 91205
77 92562
80 91352
86 92354
87 92506
91 94015
92 33149
96 91801
94 90813
97 95815
99 92139
100 93033
101 95758
106 91740
110 94509
113 92126
114 95065
115 91750
116 90066
122 91950
123 93277
124 90059
127 94585
130 90015
133 93003
136 74105
137 92103
138 92585
140 92376
152 91709
157 95841
158 90250
160 90248
163 90007
165 90302
167 95336
164 90405
169 92345
166 92101
173 91326
174 93728
186 91214
187 90029
188 90036
191 91405
194 91950
196 90402
197 91911
198 91402
206 90802
208 92507
216 95621
212 90029
217 90247
220 91405
222 92882
225 92802
226 90706
227 90706
230 93291
233 90012
234 90065
240 91945
241 95670
242 92553
244 90057
245 93704
247 91978
252 93640
256 95620
258 90807
259 92113
263 92231
266 91768
267 91773
268 95

In [10]:
df.tail(50)

,Last Name,First Name,Middle Name,A/K/A-Also Known As\nD/B/A-Doing Business as,Address(es),Provider Type,License Number,Provider Number,Date of Suspension,Active Period,1st Address,State Code,City,Zip Code,Street Address
21586,Zrodlowski,John,Edward,NaN,"2351 Shamrock St., San Diego, CA",Registered Nurse,562829,NaN,11/12/2009,Indefinitely effective,"2351 Shamrock St., San Diego, CA",CA,San Diego,92105,2351 Shamrock St.
21587,Zsiba,Cindy,Ann,"Curtis, Cindy Ann; Harrison, Cindy Ann; Zsiba,...","4760 West Belmont Ave, Fresno, CA, 93722",Registered Nurse,430930,NaN,02/20/2015,Indefinitely effective,"4760 West Belmont Ave, Fresno, CA, 93722",CA,Fresno,93722,4760 West Belmont Ave
21588,Zucker,Martin,NaN,NaN,"401 Pellertier Dr., Sioux City, IA",Physician,CFE13040,00C130400,12/11/1995,Indefinitely effective,"401 Pellertier Dr., Sioux City, IA",IA,Sioux City,Unknown,401 Pellertier Dr.
21589,Zulauf,Rebecca,Ann,"Nichols, Rebecca; Knodle, Rebecca; Newman, Reb...","10504 Santana St., Santee, CA, 92071",Registered Nurse,503637,NaN,04/03/2018,Indefinitely effective,"10504 Santana St., Santee, CA, 92071",CA,Santee,92071,10504 Santana St.
21590,Zuniga,Alexis,NaN,NaN,"2070 Lucretia Ave., No. 101, San Jose, CA, 95122",Pharmacy Technician,130272,NaN,09/17/2017,Indefinitely effective,"2070 Lucretia Ave., No. 101, San Jose, CA, 95122",CA,San Jose,95122,2070 Lucretia Ave.
21591,Zuniga,Bonnie,NaN,NaN,"P.O. Box 1747, Morgan Hill, CA, 95038",In-Home Supportive Services Provider,NaN,NaN,03/23/2016,Indefinitely effective,"P.O. Box 1747, Morgan Hill, CA, 95038",CA,Morgan Hill,95038,P.O. Box 1747
21592,Zuniga,Guadalupe,Esther,NaN,"3800 Bradford St., La Verne, CA, 91750",Registered Nurse,390451,NaN,07/08/2016,Indefinitely effective,"3800 Bradford St., La Verne, CA, 91750",CA,La Verne,91750,3800 Bradford St.
21593,Zuniga,Luis,Aroldo,NaN,"6244 Wittenham Way, Orangevale, CA",Respiratory Care Practitioner,14214,NaN,6/19/2006,Indefinitely effective,"6244 Wittenham Way, Orangevale, CA",CA,Orangevale,Unknown,6244 Wittenham Way
21594,Zuvic,Alison,Marie,"Brown, Alison Marie","1409 W. 17th Street, San Pedro, CA, 90732",Registered Nurse,552572,NaN,03/05/2020,Indefinitely effective,"1409 W. 17th Street, San Pedro, CA, 90732",CA,San Pedro,90732,1409 W. 17th Street
21595,Zverev,Emilia,E.,"Zverev, Emiliya; Fishman, Emiliya","6731 Ventura Canyon Ave., Van Nuys, CA, 91401","Biller, Medical Transportation",NaN,NaN,05/03/2016,Indefinitely effective,"6731 Ventura Canyon Ave., Van Nuys, CA, 91401",CA,Van Nuys,91401,6731 Ventura Canyon Ave.


### I reduced the number of 'Unknown' zip codes to just 3,190 from 11,876 before using the API, greatly enhancing my ability to accurately visualize the geographical distribution of suspended providers.

### Now, I would like to simplify the Provider Type field to be able to more accurately visualize which types of providers are being suspended.

As with the addresses, there are Provider Type values with multiple entries for some rows, and as with the addresses I have decided to only take the first provider type entered for any rows with multiple provider types.

In [12]:
df['Provider Type'] = df['Provider Type'].apply(lambda x: str(x).split(',')[0].split(';')[0])
df

,Last Name,First Name,Middle Name,A/K/A-Also Known As\nD/B/A-Doing Business as,Address(es),Provider Type,License Number,Provider Number,Date of Suspension,Active Period,1st Address,State Code,City,Zip Code,Street Address
0,111 Pharmacy,NaN,NaN,NaN,"111 West Beverly Blvd., Ste. B, Montebello, CA...",Pharmacy/Pharmacist,41023,"PHA410230, PHA393440, 1821137118",7/8/2016,Indefinitely effective,"111 West Beverly Blvd., Ste. B, Montebello, CA...",CA,Montebello,90640,111 West Beverly Blvd.
1,"2nd Chance for Recovery, Inc.",NaN,NaN,NaN,"600 East 7th St., Los Angeles, CA, 90021",Entity/Corporation,NaN,"1003077710, 1033522974, 1346589777, 1760751606",05/23/2018,Indefinitely effective,"600 East 7th St., Los Angeles, CA, 90021",CA,Los Angeles,90021,600 East 7th St.
2,3 Angeles Medical Clinic,NaN,NaN,NaN,"333 Wilkerson Ave., Stes. B & C, Perris, CA; 4...",Clinic,NaN,NaN,3/11/2009,Indefinitely effective,"333 Wilkerson Ave., Stes. B & C, Perris, CA",CA,Perris,92570,333 Wilkerson Ave.
3,52 Street Pharmacy,NaN,NaN,NaN,"5231 El Cajon Blvd., San Diego, CA,",Pharmacy,NaN,"PHA118810, PHA200040, PHA217110, CGP14678",12/26/1990,Indefinitely effective,"5231 El Cajon Blvd., San Diego, CA",CA,San Diego,92115,5231 El Cajon Blvd.
4,A & R Medical Transport,NaN,NaN,"Aghajanyan, Mike; Aghajanyan, Roland; Aghajany...","5300 Santa Monica Blvd., Ste. #216, Los Angele...",Medical Transportation/Owner,NaN,MTN01203F,7/4/2012,Indefinitely effective,"5300 Santa Monica Blvd., Ste. #216, Los Angele...",CA,Los Angeles,90029,5300 Santa Monica Blvd.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21631,Wehrly,John,Dale,NaN,"154 Ely Street, Oceanside, CA, 92054",Registered Nurse,484699,NaN,06/15/2023,Indefinitely effective,"154 Ely Street, Oceanside, CA, 92054",CA,Oceanside,92054,154 Ely Street
21632,Weinberger,Alan,Walter,NaN,"P.O. Box 676, South Pasadena, CA, 91031",Physician,G 33359,"00G333590, 1124015011",05/18/2023,Indefinitely effective,"P.O. Box 676, South Pasadena, CA, 91031",CA,South Pasadena,91031,P.O. Box 676
21633,Winters,Julie,Mae,NaN,"6024 Fyler Avenue, Saint Louis, MO, 63139",Registered Nurse,774207,NaN,06/16/2023,Indefinitely effective,"6024 Fyler Avenue, Saint Louis, MO, 63139",MO,Saint Louis,63139,6024 Fyler Avenue
21634,Dondiego,Madelaine,NaN,NaN,"Delano, CA",In-Home Supportive Services Provider,1336006,NaN,07/11/2023,Indefinitely effective,"Delano, CA",CA,Delano,Unknown,Delano


### Adding Zip Code Coordinates
I now read in a CSV file that has lattitude and longitude coordinates for every zip code in North America and merge it with my final provider dataset to create a new dataset with a total of 18,114 rows that will be used for our geographical visualization, excluding the remaining 3,190 rows with 'Unknown' zip codes by using an inner join. Having two seperate datasets avoids issues arising from rows with null values in the geographic visualization.  


In [13]:
zip_latlong = pd.read_csv('zip_to_lat_lon_North America.csv', dtype={'postal code': str})
zip_latlong['postal code'] = zip_latlong['postal code'].astype(str).str.zfill(5)

zip_latlong = zip_latlong[zip_latlong['country code'] == 'US']
zip_latlong = zip_latlong[['postal code','state','province_or_county','latitude','longitude']].reset_index(drop = True)
# #zip_latlong.columns = [str(col) for col in zip_latlong.columns]

zip_latlong

C:\Users\Hernando\AppData\Local\Temp\ipykernel_8112\3375427282.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  zip_latlong = pd.read_csv('zip_to_lat_lon_North America.csv', dtype={'postal code': str})


,postal code,state,province_or_county,latitude,longitude
0,99553,Alaska,Aleutians East,54.1430,-165.7854
1,99571,Alaska,Aleutians East,55.1858,-162.7211
2,99583,Alaska,Aleutians East,54.8542,-163.4113
3,99612,Alaska,Aleutians East,55.0628,-162.3056
4,99661,Alaska,Aleutians East,55.3192,-160.4914
...,...,...,...,...,...
41465,96558,NaN,NaN,19.7542,-155.5858
41466,96598,NaN,NaN,-89.9976,139.2729
41467,96599,NaN,NaN,-77.8460,166.6760
41468,96860,NaN,NaN,21.3448,-157.9774


In [14]:
df['postal code'] = df['Zip Code']

# Convert to string and standardize format (if necessary)
df['postal code'] = df['postal code'].astype(str)
zip_latlong['postal code'] = zip_latlong['postal code'].astype(str)

# Now perform the merge
merged_df = df.merge(zip_latlong, left_on='postal code', right_on='postal code', how='inner')
merged_df

,Last Name,First Name,Middle Name,A/K/A-Also Known As\nD/B/A-Doing Business as,Address(es),Provider Type,License Number,Provider Number,Date of Suspension,Active Period,1st Address,State Code,City,Zip Code,Street Address,postal code,state,province_or_county,latitude,longitude
0,111 Pharmacy,NaN,NaN,NaN,"111 West Beverly Blvd., Ste. B, Montebello, CA...",Pharmacy/Pharmacist,41023,"PHA410230, PHA393440, 1821137118",7/8/2016,Indefinitely effective,"111 West Beverly Blvd., Ste. B, Montebello, CA...",CA,Montebello,90640,111 West Beverly Blvd.,90640,California,Los Angeles,34.0133,-118.1130
1,Bonilla,Blanca,NaN,NaN,"828 South Taylor Ave., Unit #5, Montebello, CA",Certified Nurse Assistant,337480,NaN,12/20/2005,Indefinitely effective,"828 South Taylor Ave., Unit #5, Montebello, CA",CA,Montebello,90640,828 South Taylor Ave.,90640,California,Los Angeles,34.0133,-118.1130
2,Cal Med Health Plan Inc,NaN,NaN,NaN,"2457 West Beverly Blvd., Montebello, CA; 805 V...",Corporation Owner,NaN,NaN,2/11/2009,Indefinitely effective,"2457 West Beverly Blvd., Montebello, CA",CA,Montebello,90640,2457 West Beverly Blvd.,90640,California,Los Angeles,34.0133,-118.1130
3,Chavez,Linda,Martinez,NaN,"1791 Neil Armstrong St., Unit #207, Montebello...",Employee,NaN,NaN,5/20/2003,Indefinitely effective,"1791 Neil Armstrong St., Unit #207, Montebello...",CA,Montebello,90640,1791 Neil Armstrong St.,90640,California,Los Angeles,34.0133,-118.1130
4,Gomez,Fabian,NaN,NaN,"711 West Whittier Blvd., Montebello, CA",Physician,NaN,"00C358860, 00C358861",6/6/1994,Indefinitely effective,"711 West Whittier Blvd., Montebello, CA",CA,Montebello,90640,711 West Whittier Blvd.,90640,California,Los Angeles,34.0133,-118.1130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18109,Higgins,Pamela,Kay,NaN,"4770 Pedro Hill Road, Pilot Hill, CA, 95664",Nurse Midwife Furnishing,"358140, 18873, 644, 644",1750419008,06/14/2023,Indefinitely effective,"4770 Pedro Hill Road, Pilot Hill, CA, 95664",CA,Pilot Hill,95664,4770 Pedro Hill Road,95664,California,El Dorado,38.8135,-121.0308
18110,Holt,Joshua,Allen,NaN,"10998 SW Celeste Lane, Apt. 103, Portland, OR,...",Registered Nurse,837298,NaN,06/06/2023,Indefinitely effective,"10998 SW Celeste Lane, Apt. 103, Portland, OR,...",SW,Portland,97225,10998 SW Celeste Lane,97225,Oregon,Washington,45.4985,-122.7787
18111,Lipton,Lisa,Renee,"Weisz, Lisa Renee;\nWeisz Lipton, Lisa Renee","9231 Linkmeadow Lane, Houston, TX, 77025",Other,94026573,NaN,06/23/2023,Indefinitely effective,"9231 Linkmeadow Lane, Houston, TX, 77025",TX,Houston,77025,9231 Linkmeadow Lane,77025,Texas,Harris,29.6889,-95.4341
18112,Weinberger,Alan,Walter,NaN,"P.O. Box 676, South Pasadena, CA, 91031",Physician,G 33359,"00G333590, 1124015011",05/18/2023,Indefinitely effective,"P.O. Box 676, South Pasadena, CA, 91031",CA,South Pasadena,91031,P.O. Box 676,91031,California,Los Angeles,34.1161,-118.1503


### Finally, I filter out a row from both data sets that I noticed when visualizing suspensions over time that showed the impossible year of 1753 for the 'Date of Suspension' 

In [15]:
#filtering out row that has year of suspension as 1753
df['Date of Suspension'] = pd.to_datetime(df['Date of Suspension'])
df['Year'] = df['Date of Suspension'].dt.year
df = df[df['Year'] != 1753]

merged_df['Date of Suspension'] = pd.to_datetime(merged_df['Date of Suspension'])
merged_df['Year'] = merged_df['Date of Suspension'].dt.year
merged_df = merged_df[merged_df['Year'] != 1753]

In [16]:
merged_df

,Last Name,First Name,Middle Name,A/K/A-Also Known As\nD/B/A-Doing Business as,Address(es),Provider Type,License Number,Provider Number,Date of Suspension,Active Period,...,State Code,City,Zip Code,Street Address,postal code,state,province_or_county,latitude,longitude,Year
0,111 Pharmacy,NaN,NaN,NaN,"111 West Beverly Blvd., Ste. B, Montebello, CA...",Pharmacy/Pharmacist,41023,"PHA410230, PHA393440, 1821137118",2016-07-08,Indefinitely effective,...,CA,Montebello,90640,111 West Beverly Blvd.,90640,California,Los Angeles,34.0133,-118.1130,2016.0
1,Bonilla,Blanca,NaN,NaN,"828 South Taylor Ave., Unit #5, Montebello, CA",Certified Nurse Assistant,337480,NaN,2005-12-20,Indefinitely effective,...,CA,Montebello,90640,828 South Taylor Ave.,90640,California,Los Angeles,34.0133,-118.1130,2005.0
2,Cal Med Health Plan Inc,NaN,NaN,NaN,"2457 West Beverly Blvd., Montebello, CA; 805 V...",Corporation Owner,NaN,NaN,2009-02-11,Indefinitely effective,...,CA,Montebello,90640,2457 West Beverly Blvd.,90640,California,Los Angeles,34.0133,-118.1130,2009.0
3,Chavez,Linda,Martinez,NaN,"1791 Neil Armstrong St., Unit #207, Montebello...",Employee,NaN,NaN,2003-05-20,Indefinitely effective,...,CA,Montebello,90640,1791 Neil Armstrong St.,90640,California,Los Angeles,34.0133,-118.1130,2003.0
4,Gomez,Fabian,NaN,NaN,"711 West Whittier Blvd., Montebello, CA",Physician,NaN,"00C358860, 00C358861",1994-06-06,Indefinitely effective,...,CA,Montebello,90640,711 West Whittier Blvd.,90640,California,Los Angeles,34.0133,-118.1130,1994.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18109,Higgins,Pamela,Kay,NaN,"4770 Pedro Hill Road, Pilot Hill, CA, 95664",Nurse Midwife Furnishing,"358140, 18873, 644, 644",1750419008,2023-06-14,Indefinitely effective,...,CA,Pilot Hill,95664,4770 Pedro Hill Road,95664,California,El Dorado,38.8135,-121.0308,2023.0
18110,Holt,Joshua,Allen,NaN,"10998 SW Celeste Lane, Apt. 103, Portland, OR,...",Registered Nurse,837298,NaN,2023-06-06,Indefinitely effective,...,SW,Portland,97225,10998 SW Celeste Lane,97225,Oregon,Washington,45.4985,-122.7787,2023.0
18111,Lipton,Lisa,Renee,"Weisz, Lisa Renee;\nWeisz Lipton, Lisa Renee","9231 Linkmeadow Lane, Houston, TX, 77025",Other,94026573,NaN,2023-06-23,Indefinitely effective,...,TX,Houston,77025,9231 Linkmeadow Lane,77025,Texas,Harris,29.6889,-95.4341,2023.0
18112,Weinberger,Alan,Walter,NaN,"P.O. Box 676, South Pasadena, CA, 91031",Physician,G 33359,"00G333590, 1124015011",2023-05-18,Indefinitely effective,...,CA,South Pasadena,91031,P.O. Box 676,91031,California,Los Angeles,34.1161,-118.1503,2023.0


### The two datasets are ready to be saved and read into my Streamlit app for interactive filtering and visualization tasks

In [17]:
merged_df.to_csv('banned_providers_merged_lat.csv', index=False)
df.to_csv('banned_providers_final.csv', index=False)

The number of unknown zip codes before querying Nomatim's API, and the number after:

In [11]:
print(len(unknown_zip_df))

unknown_zip_df2 = df[df['Zip Code'] == 'Unknown']
print(len(unknown_zip_df2))


11876
3190
